In [1]:
!pip install selenium

In [1]:
import pandas as pd
import numpy as np

In [65]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome("/chromedriver")
driver.get("https://www.spglobal.com/esg/scores/")
time.sleep(3)
click_accept = driver.find_elements(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
click_accept[0].click()

historical_scores = {}
company_data = {}

companies = {
    "AAPL": "Apple Inc.",
    "AMZN": "Amazon.com, Inc.",
    "BRK": "Berkshire Hathaway Inc.",
    "CVX": "Chevron Corporation",
    "GOOGL": "Alphabet Inc.",
    "HD": "The Home Depot, Inc.",
    "JNJ": "Johnson & Johnson",
    "MRK": "Merck & Co., Inc.",
    "MSFT": "Microsoft Corporation",
    "NVDA": "NVIDIA Corporation",
    "PG": "The Procter & Gamble Company",
    "TSLA": "Tesla, Inc.",
    "UNH": "UnitedHealth Group Incorporated",
    "V": "Visa Inc.",
    "XOM": "Exxon Mobil Corporation"
}


i = 1
for ticker, name in companies.items():
    if i == 1:
        search_box = driver.find_elements(By.XPATH, "/html/body/div[3]/div[11]/div/div[3]/div/div/div[2]/div[1]/input")
        search_box[0].send_keys(name)
    else:
        search_box = driver.find_elements(By.XPATH, "/html/body/div[3]/div[1]/div[3]/div/div/div[2]/div[1]/input")
        search_box[0].send_keys(name)
    time.sleep(1)
    search_box[0].send_keys(Keys.ENTER)
    time.sleep(5)

    try:
        ESG_score = driver.find_elements(By.XPATH, '//*[@id="esg-score"]')[0].text
        industry = driver.find_elements(By.XPATH, '//*[@id="company-industry"]')[0].text
        company_data[ticker] = industry
        time.sleep(3)

        historical_scores[ticker] = {}
        historical_scores[ticker]['ESG Scores'] = []
        historical_scores[ticker]['Company Performance'] = []
        historical_scores[ticker]['Industry Best Performance'] = []
        historical_scores[ticker]['Industry Mean Performance'] = []

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        historical_chart = soup.find('g', {'class': 'highcharts-markers highcharts-series-0 highcharts-line-series highcharts-tracker'})
        points = historical_chart.find_all('path', {'class':'highcharts-point'})
        for point in points:
            label = point.get('aria-label')
            historical_scores[ticker]['ESG Scores'].append(label)

        company_perf = soup.find('g', {'class' : 'highcharts-markers highcharts-series-0 highcharts-area-series highcharts-tracker'})
        company_points = company_perf.find_all('path', {'class': 'highcharts-point'})
        for point in company_points:
            label = point.get('aria-label')
            historical_scores[ticker]['Company Performance'].append(label)

        industry_mean = soup.find('g', {'class' : 'highcharts-markers highcharts-series-2 highcharts-line-series highcharts-tracker'})
        industry_mean_points = industry_mean.find_all('path', {'class': 'highcharts-point'})
        for point in industry_mean_points:
            label = point.get('aria-label')
            historical_scores[ticker]['Industry Mean Performance'].append(label)

        industry_best = soup.find('g', {'class' : 'highcharts-markers highcharts-series-1 highcharts-line-series highcharts-tracker'})
        industry_best_points = industry_best.find_all('path', {'class': 'highcharts-point'})
        for point in industry_best_points:
            label = point.get('aria-label')
            historical_scores[ticker]['Industry Best Performance'].append(label)

    except:
        if i == 1:
            search_box = driver.find_elements(By.XPATH, "/html/body/div[3]/div[11]/div/div[3]/div/div/div[2]/div[1]/input")
            search_box[0].clear()
        else:
            search_box = driver.find_elements(By.XPATH, "/html/body/div[3]/div[1]/div[3]/div/div/div[2]/div[1]/input")
            search_box[0].clear()
        print(f"unable to scrape {ticker}")
    i += 1

driver.close()

/var/folders/27/xh3tm0h964n5gjzv1brx5vrc0000gn/T/ipykernel_37980/50295122.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/chromedriver")


unable to scrape JNJ


In [67]:
company_data

{'AAPL': 'Industry: THQ Computers & Peripherals and Office Electronics',
 'AMZN': 'Industry: RTS Retailing',
 'BRK': 'Industry: FBN Diversified Financial Services and Capital Markets',
 'CVX': 'Industry: OGX Oil & Gas Upstream & Integrated',
 'GOOGL': 'Industry: IMS Interactive Media, Services & Home Entertainment',
 'HD': 'Industry: RTS Retailing',
 'MRK': 'Industry: DRG Pharmaceuticals',
 'MSFT': 'Industry: SOF Software',
 'NVDA': 'Industry: SEM Semiconductors & Semiconductor Equipment',
 'PG': 'Industry: HOU Household Products',
 'TSLA': 'Industry: AUT Automobiles',
 'UNH': 'Industry: HEA Health Care Providers & Services',
 'V': 'Industry: TSV IT services',
 'XOM': 'Industry: OGX Oil & Gas Upstream & Integrated'}

In [68]:
# Helper Function to Process Scrapped Data
def process_historical_scores(scores):
    """
    Takes in scrapped data and extract Historical ESG Scores, Company Performance in various categories,
    and Industry Performances (Mean and Best) in the same categories.
    """
    ESG_scores = {}
    company_performances = {}

    for ticker, scrapped_data in scores.items():
        ESG_scores[ticker] = {}
        company_performances[ticker] = {}

        for data in scrapped_data['ESG Scores']:
            split = data.split(", ")
            year = split[0].split(". ")[-1]
            esg_score = split[-1].split(".")[0]
            ESG_scores[ticker][int(year)] = int(esg_score)

        for data in scrapped_data['Company Performance']:
            if "/ " in data: # to handle "Information Security/ Cybersecurity & System Availability"
                data = data.replace("/ ", " ")
            split = data.split(", ")
            category = split[0].split(". ")[-1]
            perf_score = split[-1].split(".")[0]
            if category not in company_performances[ticker].keys():
                company_performances[ticker][category] = {}    
            company_performances[ticker][category]['Company'] = int(perf_score)

        for data in scrapped_data['Industry Best Performance']:
            if "/ " in data: # to handle "Information Security/ Cybersecurity & System Availability"
                data = data.replace("/ ", " ")
            split = data.split(", ")
            category = split[0].split(". ")[-1]
            perf_score = split[-1].split(".")[0]
            if category not in company_performances[ticker].keys():
                company_performances[ticker][category] = {}  
            company_performances[ticker][category]['Industry Best'] = int(perf_score)

        for data in scrapped_data['Industry Mean Performance']:
            if "/ " in data: # to handle "Information Security/ Cybersecurity & System Availability"
                data = data.replace("/ ", " ")
            split = data.split(", ")
            category = split[0].split(". ")[-1]
            perf_score = split[-1].split(".")[0]
            if category not in company_performances[ticker].keys():
                company_performances[ticker][category] = {}  
            company_performances[ticker][category]['Industry Mean'] = int(perf_score)

    return ESG_scores, company_performances

In [70]:
ESG_scores, company_performances = process_historical_scores(historical_scores)

In [92]:
performance_dict = {}
for ticker, data in company_performances.items():
    performance_dict[ticker] = pd.DataFrame.from_dict(data, orient='index')

In [93]:
esg_df = pd.DataFrame.from_dict(ESG_scores)

In [96]:
esg_df

,AAPL,AMZN,BRK,CVX,GOOGL,HD,MRK,MSFT,NVDA,PG,TSLA,UNH,V,XOM
2018,27,14,10,43,31,30,39,58,69,52,13,76,53,35
2019,29,18,9,40,38,31,40,57,72,60,14,69,58,37
2020,29,21,8,42,40,26,39,58,74,60,15,70,63,36
2021,32,24,10,39,44,37,38,58,74,58,27,74,62,36
2022,37,22,12,43,46,33,43,56,72,49,37,79,65,37


In [94]:
performance_dict['V']

,Company,Industry Best,Industry Mean
Human Capital Development,68,98,35
Information Security Cybersecurity & System Availability,73,100,26
Innovation Management,26,100,12
Operational Eco-Efficiency,95,100,25
Talent Attraction & Retention,58,90,16
Climate Strategy,88,99,18
Environmental Policy & Management Systems,36,100,25
